In [1]:
from sklearn.ensemble import RandomForestClassifier

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures


from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.pipeline import make_pipeline

%matplotlib inline

from sklearn.model_selection import train_test_split



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)


## Random forest

In [ ]:
model3 = RandomForestClassifier(n_estimators=20)

model3.fit(X_train, y_train)


## Lasso

In [ ]:

perform = []
lambdas = np.logspace(-4,4,33)

for lambda_ in lambdas:
    pipe_lasso = Lasso(alpha=lambda_, random_state=1)
    pipe_lasso.fit(X_train_std, y_train)
    perform.append([lambda_, np.sqrt(mse(pipe_lasso.predict(X_train_std),y_train)), 
                    np.sqrt(mse(pipe_lasso.predict(X_test_std),y_test))])
    
pf = pd.DataFrame(perform)
headers = ['lambda', 'RMSE_train', 'RMSE_test']
pf.columns = headers
pf.set_index('lambda')


## kFold

In [ ]:
from sklearn.model_selection import KFold

X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

kfolds = KFold(n_splits=5)

mseCV = []

lambdas =  np.logspace(-5, 4, 12)

for lambda_ in lambdas:
   
    pipe_lassoCV = make_pipeline(PolynomialFeatures(include_bias=False),
                                 StandardScaler(),
                                 Lasso(alpha=lambda_, random_state=1))    
    mseCV_ = []
    
    for train_idx, val_idx in kfolds.split(X_dev, y_dev):
        X_train, y_train = X_dev[train_idx], y_dev[train_idx]
        X_val, y_val = X_dev[val_idx], y_dev[val_idx] 

        pipe_lassoCV.fit(X_train, y_train)
        
        msetest = mse(pipe_lassoCV.predict(X_test), y_test)
        mseCV_.append(msetest)    
    mseCV.append(np.mean(mseCV_))

optimal = pd.Series(mseCV,index=lambdas).nsmallest(1)
print("Optimal lambda")
print("lambda"," "*5, "Mean MSE")
print(optimal)
